# 4. Feature Extraction


## 4.1 Meaning of Feature Extraction


It means producing variables from raw data.You can think in two scope.
1. Derive variables from structural data
    - Producing variables through existing variables.
2. Derive variables from non-structural data
    - It means producing variables from data such as image, sound, writing.

**Why do we need such a thing?**
- We are working on numerical data.We use non-structural things to transform them into mathematically agreement.

The application of feature extraction to a long text provides work with natural language processing.Feature Extraction allows us to create a variable that can be expressed as numerical from a non -structural data by calculating how much each word is used in the sentence and giving a weight to each.

------

Ham veriden değişken üretmek demektir. İki kapsamda düşünebilir.
1. Yapısal verilerden değişkenler türetmek
    - Mevcut değişkenler üzerinden değişkenler üretmek anlamına gelmektedir.
2. Yapısal olmayan verilerden değişkenler türetmek
    - Görüntü, ses, yazı gibi verilerden değişkenler üretmek anlamına gelmektedir.

**Neden böyle bir şeye ihtiyaç duyarız?**
- Numerik veriler üzerinde çalışıyoruz. Yapısal olmayan şeyleri matematiksel olarak anlaşabilecek şeylere dönüştürmek için kullanırız. 


| Timestamp        	|
|------------------	|
| 2021-02-05 07:45 	|
| 2021-02-04 21:05 	|
| 2019-05-17 09:51 	|
| 2019-05-16 21:27 	|
| 2019-05-16 13:40 	|

**Bu veriden nasıl bir özellik çıkarımı yapılabilir?**

| Timestamp        	| Year 	| Month 	| Day 	| Hour 	| Day Name 	|
|------------------	|------	|-------	|-----	|------	|----------	|
| 2021-02-05 07:45 	| 2021 	| 2     	| 5   	| 7    	| Friday   	|
| 2021-02-04 21:05 	| 2021 	| 2     	| 4   	| 21   	| Thursday 	|
| 2019-05-17 09:51 	| 2021 	| 5     	| 17  	| 9    	| Friday   	|
| 2019-05-16 21:27 	| 2019 	| 5     	| 16  	| 21   	| Thursday 	|
| 2019-05-16 13:40 	| 2019 	| 5     	| 16  	| 13   	| Thursday 	|


Feature Extraction işlemini uzun bir metine uygulanması Doğal Dil İşleme ile çalışmayı sağlamaktadır. Feature Extraction her bir kelimenin cümle içerisinde ne kadar kullanıldığının hesaplanması ve her bir birine bir ağırlık vermesi ile yapısal olmayan bir veriden numerik olarak ifade edilebilecek bir değişken oluşturmamızı sağlar. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [3]:
# Data set for large scale application
def load_application_train():
    data = pd.read_csv("datasets/application_train2.csv")
    return data


df_application = load_application_train()
df_application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,351000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.019,-9461,-637,-3648.000,-2120,NaN,1,1,0,1,1,0,Laborers,1.000,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083,0.263,0.139,0.025,0.037,0.972,0.619,0.014,0.000,0.069,0.083,0.125,0.037,0.020,0.019,0.000,0.000,0.025,0.038,0.972,0.634,0.014,0.000,0.069,0.083,0.125,0.038,0.022,0.020,0.000,0.000,0.025,0.037,0.972,0.624,0.014,0.000,0.069,0.083,0.125,0.037,0.021,0.019,0.000,0.000,reg oper account,block of flats,0.015,"Stone, brick",No,2.000,2.000,2.000,2.000,-1134.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,1129500.000,Family,State servant,Higher education,Married,House / apartment,0.004,-16765,-1188,-1186.000,-291,NaN,1,1,0,1,1,0,Core staff,2.000,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311,0.622,NaN,0.096,0.053,0.985,0.796,0.060,0.080,0.035,0.292,0.333,0.013,0.077,0.055,0.004,0.010,0.092,0.054,0.985,0.804,0.050,0.081,0.035,0.292,0.333,0.013,0.079,0.055,0.000,0.000,0.097,0.053,0.985,0.799,0.061,0.080,0.035,0.292,0.333,0.013,0.079,0.056,0.004,0.010,reg oper account,block of flats,0.071,Block,No,1.000,0.000,1.000,0.000,-828.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,135000.000,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010,-19046,-225,-4260.000,-2531,26.000,1,1,1,1,1,0,Laborers,1.000,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.556,0.730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
# Data set for small scale application
def load_titanic():
    data = pd.read_csv("datasets/titanic.csv")
    return data


df_titanic = load_titanic()
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [5]:
def grab_col_names(dataframe, categorical_threshold = 10, cardinal_threshold= 20):
    """
    It gives the names of categorical, numerical and categorical but cardinal variables in the data set.
    Note: Categorical variables but numerical variables are also included in categorical variables.

    Parameters
    ----------
    dataframe : dataframe
        The dataframe from which variables names are to be retrieved.
    categorical_threshold : int, optional
        class threshold for numeric but categorical variables
    cardinal_threshold : int, optional
        Class threshold for categorical but cardinal variables

    Returns
    -------
        categorical_cols : list
            Categorical variable list
        numerical_cols : list
            Numerical variable list
        cardinal_cols : list
            Categorical looking cardinal variable list

    Examples
    -------
        import seaborn as sns
        df = sns.load_titanic_dataset("iris")
        print(grab_col_names(df))

    Notes
    -------
        categorical_cols + numerical_cols + cardinal_cols = total number of variables.
        nominal_cols is inside categorical_cols
        The sum of the 3 returned lists equals the total number of variables: categorical_cols + cardinal_cols = number of variables

    """

    categorical_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    nominal_cols = [col for col in dataframe.columns if dataframe[col].nunique() < categorical_threshold and dataframe[col].dtypes != "O"]
    cardinal_cols = [col for col in dataframe.columns if dataframe[col].nunique() > cardinal_threshold and dataframe[col].dtypes == "O"]
    categorical_cols = categorical_cols + nominal_cols
    categorical_cols = [col for col in categorical_cols if col not in cardinal_cols]

    # numerical_cols
    numerical_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    numerical_cols = [col for col in numerical_cols if col not in categorical_cols]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'categorical_cols: {len(categorical_cols)}')
    print(f'numerical_cols: {len(numerical_cols)}')
    print(f'cardinal_cols: {len(cardinal_cols)}')
    print(f'nominal_cols: {len(nominal_cols)}')
    return categorical_cols, numerical_cols, cardinal_cols, nominal_cols

## 4.2 Binary Features

In [6]:
df_titanic["NEW_CABIN_BOOL"] = df_titanic["Cabin"].notnull().astype("int")

In [7]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0


In [8]:
df_titanic.groupby("NEW_CABIN_BOOL").agg({"Survived":"mean"})

# 0 without a cabin number, 1 with cabin number
# Inference: Those with a cabin number have a higher chance of survival.

,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


In [9]:
from statsmodels.stats.proportion import proportions_ztest

test_stat, pvalue = proportions_ztest(count=[df_titanic.loc[df_titanic["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
                                             df_titanic.loc[df_titanic["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],

                                      nobs=[df_titanic.loc[df_titanic["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
                                            df_titanic.loc[df_titanic["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 9.4597, p-value = 0.0000


The Z test tells us that there is no difference between P1 and P1.This indicates that there is no difference between the cabin number and the non -without.If the P-Value value is less than 0.5, we prove that there is a difference in this case.The fact that it is close to 0 states that there is a serious difference and that this data is of serious importance in the event that it cannot survive.However, we do not know how important it is in multiple variable analysis.So we will see if it is certainly serious.

------

Z testi bize p1 ve p1 arasında herhangi bir fark olmadığını söyler. Bu kabin numarası olan ile olmayan arasında bir fark olmadığını belirtir. Eğer p-value değeri 0.5'ten küçük ise bu durumda bir fark olduğunu ispatlarız. 0'a yakın olması ise ciddi bir farkın olduğunu ve bu verinin hayatta kalıp kalınamayacağı olayında ciddi bir öneme sahip olduğunu belirtir. Ancak çoklu değişken analizinde ne kadar öneme sahip olduğunu bilmiyoruz. Bu yüzden kesin bir ciddi öneme sahip olup olmadığına bakacağız.

In [10]:
df_titanic.loc[((df_titanic["SibSp"] + df_titanic["Parch"]) > 0), "NEW_IS_ALONE"] = "NO"
df_titanic.loc[((df_titanic["SibSp"] + df_titanic["Parch"]) == 0), "NEW_IS_ALONE"] = "YES"

In [11]:
df_titanic.groupby("NEW_IS_ALONE").agg({"Survived":"mean"})

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


In [12]:
test_stat, pvalue = proportions_ztest(count=[df_titanic.loc[df_titanic["NEW_IS_ALONE"] == "YES", "Survived"].sum(),
                                             df_titanic.loc[df_titanic["NEW_IS_ALONE"] == "NO", "Survived"].sum()],

                                      nobs=[df_titanic.loc[df_titanic["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],
                                            df_titanic.loc[df_titanic["NEW_IS_ALONE"] == "NO", "Survived"].shape[0]])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat = -6.0704, p-value = 0.0000


The H0 hypothesis indicates that there is a difference between the two.It does not indicate that this variable certainly affects the non -survival condition, but it indicates that it cannot be ignored.

------

H0 hipotezi ikisi arasında bir fark olduğunu belirtir. Hayatta kalıp kalmama durumunu bu değişkenin kesinlikle etkilediğini göstermez ama göz ardı edilemeyeceğini gösterir.

## 4.3 Text Features

In [13]:
# Letter Count

df_titanic["NEW_NAME_COUNT"] = df_titanic["Name"].str.len()
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL,NEW_IS_ALONE,NEW_NAME_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0,NO,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1,NO,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0,YES,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1,NO,44
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0,YES,24


In [14]:
# Word Count
df_titanic["NEW_NAME_WORD_COUNT"] = df_titanic["Name"].apply(lambda x: len(str(x).split(" ")))
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL,NEW_IS_ALONE,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0,NO,23,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1,NO,51,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0,YES,22,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1,NO,44,7
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0,YES,24,4


In [15]:
# Capture special structures

df_titanic["NEW_NAME_DR"] = df_titanic["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))

df_titanic.groupby("NEW_NAME_DR").agg({"Survived": ["mean","count"]})

Survived      
                mean count
NEW_NAME_DR               
0              0.383   881
1              0.500    10

## 4.4 Regex Features

In [16]:
df_titanic['NEW_TITLE'] = df_titanic.Name.str.extract(' ([A-Za-z]+)\.', expand=False)


df_titanic[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived": "mean", "Age": ["count", "mean"]})

Survived   Age       
              mean count   mean
NEW_TITLE                      
Capt         0.000     1 70.000
Col          0.500     2 58.000
Countess     1.000     1 33.000
Don          0.000     1 40.000
Dr           0.429     6 42.000
Jonkheer     0.000     1 38.000
Lady         1.000     1 48.000
Major        0.500     2 48.500
Master       0.575    36  4.574
Miss         0.698   146 21.774
Mlle         1.000     2 24.000
Mme          1.000     1 24.000
Mr           0.157   398 32.368
Mrs          0.792   108 35.898
Ms           1.000     1 28.000
Rev          0.000     6 43.167
Sir          1.000     1 49.000

## 4.5 Date Features

In [17]:
df_course_reviews = pd.read_csv("datasets/course_reviews.csv")
df_course_reviews.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000


In [18]:
df_course_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


In [19]:
df_course_reviews["Timestamp"] = pd.to_datetime(df_course_reviews["Timestamp"],format="%Y-%m-%d")

In [20]:
df_course_reviews.info() # Object tipinde datetime64 tipine döndürdük.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [21]:
# Year
df_course_reviews["year"] = df_course_reviews["Timestamp"].dt.year

# Month
df_course_reviews["month"] = df_course_reviews["Timestamp"].dt.month

In [22]:
# year_diff
df_course_reviews["year_diff"] = date.today().year - df_course_reviews["Timestamp"].dt.year

In [23]:
# month diff
df_course_reviews["month_diff"] = (date.today().year - df_course_reviews['Timestamp'].dt.year) * 12 + date.today().month - df_course_reviews['Timestamp'].dt.month

In [24]:
# day name
df_course_reviews["day_name"] = df_course_reviews["Timestamp"].dt.day_name()

In [25]:
df_course_reviews.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff,month_diff,day_name
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,1,17,Friday
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,1,17,Thursday
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,1,17,Thursday
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,1,17,Thursday
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,1,17,Thursday


## 4.6 Feature Interaction

In [26]:
df_titanic["NEW_AGE_PCLASS"] = df_titanic["Age"] * df_titanic["Pclass"]

In [27]:
df_titanic["NEW_FAMILY_SIZE"] = df_titanic["SibSp"] + df_titanic["Parch"] + 1

In [28]:
df_titanic.loc[(df_titanic['Sex'] == 'male') & (df_titanic['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'

In [29]:
df_titanic.loc[(df_titanic['Sex'] == 'male') & (df_titanic['Age'] > 21) & (df_titanic['Age'] < 50), 'NEW_SEX_CAT'] = 'maturemale'

In [30]:
df_titanic.loc[(df_titanic['Sex'] == 'male') & (df_titanic['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniormale'

In [31]:
df_titanic.loc[(df_titanic['Sex'] == 'female') & (df_titanic['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'

In [32]:
df_titanic.loc[(df_titanic['Sex'] == 'female') & (df_titanic['Age'] > 21) & (df_titanic['Age'] < 50), 'NEW_SEX_CAT'] = 'maturefemale'

In [33]:
df_titanic.loc[(df_titanic['Sex'] == 'female') & (df_titanic['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [34]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL,NEW_IS_ALONE,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR,NEW_TITLE,NEW_AGE_PCLASS,NEW_FAMILY_SIZE,NEW_SEX_CAT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0,NO,23,4,0,Mr,66.000,2,maturemale
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1,NO,51,7,0,Mrs,38.000,2,maturefemale
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0,YES,22,3,0,Miss,78.000,1,maturefemale
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1,NO,44,7,0,Mrs,35.000,2,maturefemale
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0,YES,24,4,0,Mr,105.000,1,maturemale


In [35]:
df_titanic.groupby("NEW_SEX_CAT")["Survived"].mean()

NEW_SEX_CAT
maturefemale   0.774
maturemale     0.199
seniorfemale   0.909
seniormale     0.135
youngfemale    0.679
youngmale      0.250
Name: Survived, dtype: float64